In [1]:
import numpy as np
import cupy as cu
import cupyx.scipy.ndimage

In [2]:
x_gpu = cu.array([1,2,3])

In [3]:
x_cpu = np.array([1, 2, 3])

l2_cpu = np.linalg.norm(x_cpu)

In [4]:
x_gpu = cu.array([1, 2, 3])

l2_gpu = cu.linalg.norm(x_gpu)

In [5]:
import czifile
_testFile = czifile.CziFile('sample_data/RBC_tiny.czi')

In [6]:
image  = _testFile.asarray()[0, 0, :,:,:,0]

In [7]:
import tifffile
image = tifffile.imread('sample_data/RBC_lattice.tif')

In [13]:
print(image.shape)
depth, height, original_width = image.shape


(150, 118, 209)


In [37]:

def calculate_new_dimensions(original_width, depth, shear_factor, z_pixel_size):
    # Calculate the new width by considering the maximum shear across the Z depth
    # The depth in pixels multiplied by the Z pixel size gives the total physical depth
    # Multiplying by the tangent of the excitation angle gives the total shear in microns
    # Dividing by the Y pixel size converts that shear back into Y pixels
    max_skew_in_pixels = (depth * z_pixel_size * cu.tan(angle_in_radians)) / y_pixel_size
    new_width = int(original_width + cu.abs(max_skew_in_pixels))
    return new_width


# Physical dimensions of pixels
y_pixel_size = 0.1449  # in microns
z_pixel_size = 0.3    # in microns

# Calculate the shear factor based on the physical dimensions
# We want to know how many Z pixels correspond to one Y pixel
shear_factor_per_pixel = y_pixel_size / z_pixel_size

# Convert the excitation angle to radians and calculate the shear factor
angle_in_radians = cu.radians(30)
shear_factor = cu.tan(angle_in_radians) * shear_factor_per_pixel


# Define the transformation matrix for de-skewing
# This matrix represents a shear parallel to the Y-axis
transformation_matrix = cu.array([[1, shear_factor.get(), 0],
                                  [0, 1, 0]])


# Define the transformation matrix for de-skewing each slice
transformation_matrix = cu.array([[1, shear_factor.get(), 0],
                                  [0, 1, 0]])

depth, original_height, original_width = image.shape
max_physical_skew = depth * z_pixel_size * cu.tan(angle_in_radians)
max_skew_in_pixels = max_physical_skew / y_pixel_size
new_width = int(original_width + cu.abs(max_skew_in_pixels))

# Create a new volume to hold the de-skewed images
deskeWed_volume = cu.zeros((depth, original_height, new_width), dtype=image.dtype)
image = cu.asarray(image)
for z in range(depth):
    # Calculate the slice-specific shear factor based on the Z index
    # Currently I don't think this is calculated correctly so I am using the skear_factor
    slice_shear_factor = shear_factor * z* z_pixel_size

    # Apply the 2D affine transformation to each slice along the Z-axis
    transformation_matrix = cu.array([[1, shear_factor.get(), 0],
                                  [0, 1, 0]])


    # Apply the affine transformation using the upper 2x2 matrix for each slice
    deskeWed_volume[z, :, :] = cupyx.scipy.ndimage.affine_transform(
        image[z, :, :],
        transformation_matrix[:2, :2],
        offset=0,
        output_shape=(original_height, new_width),
        order=1,
        mode='constant',
        cval=0.0)

In [38]:
print(deskeWed_volume.shape)
print(image.shape)


(150, 118, 388)
(150, 118, 209)


In [16]:
def rotate_volume(deskeWed_volume, angle_of_rotation):
    rotated_volume = cu.zeros_like(deskeWed_volume)
    
    # Rotate each slice of the volume
    for z in range(deskeWed_volume.shape[0]):
        # Rotate the 2D slice around the Z-axis (the axis perpendicular to the XY plane)
        rotated_volume[z, :, :] = cupyx.scipy.ndimage.rotate(
            deskeWed_volume[z, :, :],
            angle_of_rotation,
            axes=(1, 0),  # This rotates the slice in the XY plane
            reshape=False,
            order=1,
            mode='nearest',
            cval=0.0
        )
    
    return rotated_volume

# Apply the rotation to the de-skewed volume
angle_of_rotation = 90  # or whatever angle is needed to align with the stage coordinates
rotated_volume = rotate_volume(deskeWed_volume, angle_of_rotation)

In [39]:
import napari
views = napari.Viewer()

views.add_image(deskeWed_volume.get(), scale=(.3, .15,.15), name='deskewed')
views.add_image(image.get(), scale=(.3, .15,.15))

<Image layer 'Image' at 0x7f2bcbf36f50>

In [110]:
views.add_image(rotated_volume.get(), scale=(.3, .15,.15))

<Image layer 'Image [1]' at 0x7f57f4345810>

In [85]:
image2= tifffile.imread('sample_data/RBC_lattice.tif')
views.add_image(image2, scale=(.3, .15,.15))

<Image layer 'image2' at 0x7f58146899f0>

In [25]:
image_volume = image
angle_in_degrees = 30
y_pixel_size = 0.15
z_pixel_size = 0.3

depth, height, original_width = image_volume.shape

# Calculate the shear factor based on the excitation angle and pixel sizes
angle_in_radians = cu.radians(angle_in_degrees)
shear_factor = cu.tan(angle_in_radians) * (y_pixel_size / z_pixel_size)

# Calculate the new width to accommodate the shearing
max_skew_in_pixels = (depth - 1) * cu.tan(angle_in_radians) * (z_pixel_size / y_pixel_size)
new_width = int(original_width + cu.abs(max_skew_in_pixels))

# Create a new volume to hold the de-skewed images
deskeWed_volume = cu.zeros((depth, height, new_width), dtype=image_volume.dtype)

# Define the 4x4 affine transformation matrix for 3D shearing in y over z
transformation_matrix = cu.array([
    [1, 0, 0, 0],
    [shear_factor.get(), 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Perform the affine transformation on the entire volume
deskeWed_volume = cupyx.scipy.ndimage.affine_transform(
    image_volume,
    transformation_matrix,
    offset=0,
    output_shape=(depth, height, new_width),
    order=1,
    mode='constant',
    cval=0.0
)



In [26]:
import napari
views = napari.Viewer()
views.add_image(image_volume.get(), scale=(.3, .15,.15))
views.add_image(deskeWed_volume.get(), scale=(.3, .15,.15))

<Image layer 'Image [1]' at 0x7f2bac0bb040>